<a href="https://colab.research.google.com/github/rwcitek/StructureAndInterpretationOfComputerShells-Nushell/blob/rwc/nushell_in_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to install nushell in Jupyter

These commands have been tested in Google's CoLab and in GitHub's CodeSpaces, but may work in other Jupyter environments as well.

Display some info about who we are.

In [29]:
%%script bash
whoami
id
echo ${HOME}

codespace
uid=1000(codespace) gid=1000(codespace) groups=1000(codespace),106(ssh),107(docker),988(pipx),989(python),990(oryx),991(golang),992(sdkman),993(rvm),994(conda),995(php),996(nvs),997(nvm),998(hugo),999(dotnet)
/home/codespace


Install some tools in the environment

In [20]:
%%capture output
%%script bash
export DEBIAN_FRONTEND=noninteractive
sudo apt-get update
sudo apt-get install -y pkg-config libssl-dev jq tree vim curl less elinks


Install and configure the latest nushell.

In [21]:
%%capture output
%%script bash
cd /usr/local/bin/
elinks --dump https://github.com/nushell/nushell/releases |
  grep -m1 -o https:.*x86_64-unknown-linux-gnu.tar.gz |
  xargs curl -Ls |
  sudo tar -xzf -
sudo ln -snf ./nu-*-unknown-linux-gnu/nu
yes | nu >& /dev/null


Display the command options.  Unfortunately, only some work.

In [30]:
!nu --help

The nushell language and shell.

Usage:
  > nu {flags} (script file) ...(script args) 

Flags:
  -h, --help - Display the help message for this command
  -c, --commands <String> - run the given commands and then exit
  -e, --execute <String> - run the given commands and then enter an interactive shell
  -I, --include-path <String> - set the NU_LIB_DIRS for the given script (semicolon-delimited)
  -i, --interactive - start as an interactive shell
  -l, --login - start as a login shell
  -m, --table-mode <String> - the table mode to use. rounded is default.
  -n, --no-config-file - start with no config file and no env file
  --no-std-lib - start with no standard library
  -t, --threads <Int> - threads to use for parallel commands
  -v, --version - print the version
  --config <String> - start with an alternate config file
  --env-config <String> - start with an alternate environment config file
  --ide-goto-def <Int> - go to the definition of the item at the given position
  --ide-hover 

Display the version

In [ ]:
!nu --version

0.83.1


Get rid of the ASCII art for table display.


In [ ]:
%%script bash
sed -i -e '/mode.*rounded/ { s/mode/# mode/ ; s/$/\n    mode: none / } ' \
  ${HOME}/.config/nushell/config.nu

Verify the "table" setting has been modified in the config file.

In [22]:
!grep -C3 'mode: none' ${HOME}/.config/nushell/config.nu


    table: {
        # mode: rounded # basic, compact, compact_double, light, thin, with_love, rounded, reinforced, heavy, none, other
    mode: none 
        index_mode: always # "always" show indexes, "never" show indexes, "auto" = show indexes when a table has "index" column
        show_empty: true # show 'empty list' and 'empty record' placeholders for command output
        trim: {


## Hello, world.

In [34]:
!nu -c 'echo "Hello, world"'

Hello, world


The current date and time.

In [35]:
!nu -c 'date now'

Tue, 15 Aug 2023 00:06:13 +0000 (now)


Try piping commands into nushell.

In [1]:
%%script bash
# doesn't work
echo 'date now' | nu

Error:   × Nushell launched as a REPL, but STDIN is not a TTY; either launch in a
  │ valid terminal or provide arguments to invoke a script!



CalledProcessError: Command 'b"# doesn't work\necho 'date now' | nu\n"' returned non-zero exit status 1.

In [2]:
%%script bash
# doesn't work
echo 'date now' | nu --stdin

Error:   × Nushell launched as a REPL, but STDIN is not a TTY; either launch in a
  │ valid terminal or provide arguments to invoke a script!



CalledProcessError: Command 'b"# doesn't work\necho 'date now' | nu --stdin\n"' returned non-zero exit status 1.

Saving commands to a file and then running to a file does work.

In [3]:
%%script bash
# does work
echo 'date now' > /tmp/nushell.nu
nu /tmp/nushell.nu



Tue, 15 Aug 2023 00:15:15 +0000 (now)

## Workaround for piping

Create a script that saves stdin to a script then have nushell run the script.

In [10]:
%%script bash
cat <<'eof' | sudo tee /usr/local/bin/nushell > /dev/null
#!/usr/bin/env bash
cat > /tmp/nushell.tmp.nu
nu --config ${HOME}/.config/nushell/config.nu /tmp/nushell.tmp.nu
eof
sudo chmod +x /usr/local/bin/nushell


In [11]:
%%script nushell
ls | where type == dir

 #     name     type    size       modified    
 0   examples   dir    4.1 KB   39 minutes ago 


In [13]:
%%script nushell
echo "Hello, world"


Hello, world
